In [ ]:
import os

# 프로젝트 루트 (ship-ai)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
APP_DIR = os.path.join(BASE_DIR, "app")

# 1. app 폴더 생성
os.makedirs(APP_DIR, exist_ok=True)
print(f"✅ Created Folder: {APP_DIR}")

# 2. __init__.py (빈 파일)
with open(os.path.join(APP_DIR, "__init__.py"), "w") as f:
    pass

# 3. schemas.py (데이터 구조)
schemas_code = """
from pydantic import BaseModel
from typing import List, Optional

class PredictionRequest(BaseModel):
    ticker: str

class PredictionResponse(BaseModel):
    ticker: str
    current_price: float
    predicted_prices: List[float]
    direction: str
    analysis_comment: str
    key_factor: str
"""
with open(os.path.join(APP_DIR, "schemas.py"), "w", encoding='utf-8') as f:
    f.write(schemas_code.strip())
print("✅ Created: schemas.py")

# 4. inference.py (추론 로직 - 괴리 분석 포함)
# (주의: TimeLLM 경로는 실제 환경에 맞춰 수정될 수 있음)
inference_code = """
import torch
import numpy as np
import pandas as pd
import os
import sys

# 프로젝트 루트 경로 추가
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

# 모델 import 시도
try:
    from external.time_llm.models.TimeLLM import Model as TimeLLM
except ImportError:
    # 경로가 다를 경우를 대비해 models 폴더에서도 시도
    try:
        from models.TimeLLM import Model as TimeLLM
    except:
        print("⚠️ TimeLLM Model Import Failed. Check paths.")

class Configs:
    def __init__(self):
        self.task_name = 'long_term_forecast'
        self.is_training = 0
        self.model_id = 'Stock_Prediction'
        self.model = 'TimeLLM'
        self.seq_len = 120
        self.label_len = 60
        self.pred_len = 10
        self.enc_in = 12
        self.dec_in = 12
        self.c_out = 12
        self.llm_model = 'GPT2'
        self.llm_dim = 768
        self.llm_layers = 12
        self.patch_len = 8   # Ultimate
        self.stride = 4      # Ultimate
        self.d_model = 768   # Monster
        self.d_ff = 768      # Monster (Fixed)
        self.n_heads = 12
        self.dropout = 0.0
        self.prompt_domain = 1
        self.content = "Predict..."

def analyze_key_factors(df_window):
    # 컬럼명 매핑 (실제 CSV에 맞게 수정 필요)
    # 예: 'brent', 'usd', 'bdi', 'interest_rate' 컬럼이 있다고 가정
    # 만약 컬럼명이 다르다면 여기서 수정하세요!
    changes = {}
    
    # 예시 로직 (변동성 계산)
    target_cols = ['brent', 'usd', 'bdi'] # 주요 거시 지표
    
    for col in target_cols:
        if col in df_window.columns:
            try:
                now = df_window[col].iloc[-1]
                prev = df_window[col].iloc[-5] # 5일 전 대비
                change = (now - prev) / (prev + 1e-5)
                changes[col] = change
            except:
                pass
                
    if not changes:
        return "시장 전반", 0.0
        
    top_factor = max(changes, key=lambda k: abs(changes[k]))
    return top_factor, changes[top_factor]

class Predictor:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.configs = Configs()
        self.model = None
        self.data = None
        
    def load_model(self, model_path):
        # TimeLLM 클래스 재정의가 필요할 수 있음 (환경에 따라)
        # 여기선 위에서 import한 TimeLLM 사용
        self.model = TimeLLM(self.configs)
        state_dict = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(state_dict)
        self.model.to(self.device).float()
        self.model.eval()
        print(f"✅ Model Loaded from {model_path}")

    def load_data(self, csv_path):
        self.data = pd.read_csv(csv_path)
        # 날짜 정렬
        if 'date' in self.data.columns:
            self.data = self.data.sort_values('date')
        print(f"✅ Data Loaded: {len(self.data)} rows")

    def predict(self, ticker):
        # 데이터 필터링
        df_ticker = self.data[self.data['ticker'] == ticker].copy()
        if len(df_ticker) < 120:
            return {"error": "Not enough data"}
            
        recent_window = df_ticker.iloc[-120:]
        
        # 전처리 (Standard Scaling - 약식)
        # 실제론 학습시 사용한 Scaler를 로드해야 정확함. 여기선 구간 Scaling
        cols = [c for c in recent_window.columns if c not in ['date', 'ticker', 'ticker_name']]
        vals = recent_window[cols].values
        mean = np.mean(vals, axis=0)
        std = np.std(vals, axis=0) + 1e-5
        scaled = (vals - mean) / std
        
        batch_x = torch.tensor(scaled).unsqueeze(0).float().to(self.device)
        
        # 추론
        with torch.no_grad():
            B, Seq, C = batch_x.shape
            Pred = 10
            dummy_mark_enc = torch.zeros(B, Seq, 4).to(self.device)
            dummy_dec_in = torch.zeros(B, Pred, C).to(self.device)
            dummy_mark_dec = torch.zeros(B, Pred, 4).to(self.device)
            
            outputs = self.model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
            if isinstance(outputs, tuple): outputs = outputs[0]
            
        # 역변환
        pred_scaled = outputs[0, -Pred:, :].cpu().numpy()
        
        # 0번 컬럼이 Close라고 가정
        pred_close = (pred_scaled[:, 0] * std[0]) + mean[0]
        
        current_price = recent_window.iloc[-1]['close']
        factor_name, factor_val = analyze_key_factors(recent_window)
        
        # 코멘트 생성
        last_pred = pred_close[-1]
        ratio = (last_pred - current_price) / current_price
        
        if ratio > 0.03:
            direction = "상승"
            cmt = f"Time-LLM은 상승세(약 +{ratio*100:.1f}%)를 예측합니다."
        elif ratio < -0.03:
            direction = "하락"
            cmt = f"Time-LLM은 하락세(약 -{abs(ratio)*100:.1f}%)를 예측합니다."
        else:
            direction = "보합"
            cmt = "Time-LLM은 주가 횡보를 예측합니다."
            
        if abs(factor_val) > 0.01:
            cmt += f" (주요인: {factor_name} 변동)"
            
        return {
            "ticker": ticker,
            "current_price": float(current_price),
            "predicted_prices": pred_close.tolist(),
            "direction": direction,
            "analysis_comment": cmt,
            "key_factor": factor_name
        }

predictor = Predictor()
"""
with open(os.path.join(APP_DIR, "inference.py"), "w", encoding='utf-8') as f:
    f.write(inference_code.strip())
print("✅ Created: inference.py")

# 5. main.py (서버 진입점)
main_code = """
from fastapi import FastAPI, HTTPException
from app.schemas import PredictionRequest, PredictionResponse
from app.inference import predictor
import os

app = FastAPI(title="Ship-AI Time-LLM Engine")

@app.on_event("startup")
async def startup_event():
    print("🚀 Server Starting...")
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
    
    # 파일 경로 (환경에 맞게 수정 가능)
    MODEL_PATH = os.path.join(BASE_DIR, "models", "ship_time_llm_final.pth")
    DATA_PATH = os.path.join(BASE_DIR, "data", "processed", "final_master_table_v2.csv")
    
    if os.path.exists(MODEL_PATH):
        predictor.load_model(MODEL_PATH)
        predictor.load_data(DATA_PATH)
    else:
        print("⚠️ Model file not found. Waiting for training...")

@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    try:
        return predictor.predict(request.ticker)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
"""
with open(os.path.join(APP_DIR, "main.py"), "w", encoding='utf-8') as f:
    f.write(main_code.strip())
print("✅ Created: main.py")